In [1]:
# ===== FIXED SETUP FOR JUPYTER =====
import os
import sys

# Clear any existing Spark environment variables
spark_env_vars = ['SPARK_HOME', 'SPARK_LOCAL_DIRS', 'SPARK_CONF_DIR']
for var in spark_env_vars:
    if var in os.environ:
        del os.environ[var]

# Set clean environment
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Import and create SparkContext (only once!)
from pyspark import SparkContext, SparkConf

# Check if SparkContext already exists
try:
    # Try to access existing SparkContext
    sc.version
    print("✅ Using existing SparkContext")
except:
    # Create new SparkContext if none exists
    conf = SparkConf().setAppName("Lab2_Complete").setMaster("local[*]")
    sc = SparkContext(conf=conf)
    print("✅ Created new SparkContext")

print(f"✅ SparkContext ready - Version: {sc.version}")
print(f"✅ Application: {sc.appName}")


25/09/25 10:29:44 WARN Utils: Your hostname, thuan-precision-5560 resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
25/09/25 10:29:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/25 10:29:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/25 10:29:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/25 10:29:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/09/25 10:29:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


✅ Created new SparkContext
✅ SparkContext ready - Version: 3.5.6
✅ Application: Lab2_Complete


In [4]:
# ===== BÀI 4: ĐẾM SINH VIÊN =====
print("=== BÀI 4: ĐẾM SINH VIÊN CÓ QUÊ QUÁN Ở BÌNH ĐỊNH ===\n")

# Step 1: Examine the CSV file structure first
print("Step 1: Kiểm tra cấu trúc file CSV")

with open("../../data/SinhVien.csv", "r", encoding="utf-8") as f:
    lines = f.readlines()

print(f"✅ File có {len(lines)} dòng")
print("📄 5 dòng đầu tiên:")
for i, line in enumerate(lines[:5]):
    print(f"  {i+1}: {line.strip()}")

# Check if first line is header
first_line = lines[0].strip()
print(f"\n📊 Dòng đầu tiên: {first_line}")
if "mã_sv" in first_line.lower() or "họ_tên" in first_line:
    print("✅ Có header row")
    has_header = True
else:
    print("❌ Không có header row")
    has_header = False

=== BÀI 4: ĐẾM SINH VIÊN CÓ QUÊ QUÁN Ở BÌNH ĐỊNH ===

Step 1: Kiểm tra cấu trúc file CSV
✅ File có 2358 dòng
📄 5 dòng đầu tiên:
  1: 4351010011,Nguyễn Đình Đô,11/04/2002,Phú Yên,THPT chuyên Lê Quý Đôn,Sư phạm Toán học
  2: 4351010019,Chu Thị Kim Hà,02/09/2002,Phú Yên,THPT Phan Bội Châu,Sư phạm Toán học
  3: 4351010029,Nguyễn thị bích Hồng,12/03/2002,Quảng Ngãi,Trường THPT Lê Trung Đình,Sư phạm Toán học
  4: 4351010030,Nguyễn Thị Như Hồng,20/09/2002,Gia Lai,Trường THPT Pleiku,Sư phạm Toán học
  5: 4351010037,Võ Thụy Ngọc Huyền,09/05/2002,Bình Định,Quốc Học Quy Nhơn,Sư phạm Toán học

📊 Dòng đầu tiên: 4351010011,Nguyễn Đình Đô,11/04/2002,Phú Yên,THPT chuyên Lê Quý Đôn,Sư phạm Toán học
❌ Không có header row


In [8]:
# Step 2: Đọc file CSV bằng Spark
print("\nStep 2: Đọc file CSV bằng Spark")

# Read CSV file into RDD
students_rdd = sc.textFile("../../data/SinhVien.csv")

print(f"✅ RDD được tạo với {students_rdd.count()} dòng")
print(f"✅ Số phân vùng: {students_rdd.getNumPartitions()}")

# Filter out header if it exists
if "mã_sv" in first_line or "4351" not in first_line:
    print("✅ Loại bỏ header row")
    data_rdd = students_rdd.filter(lambda line: line != first_line)
    header = first_line
else:
    print("✅ Không có header, sử dụng tất cả dòng")
    data_rdd = students_rdd
    header = None

data_count = data_rdd.count()
print(f"✅ Số dòng dữ liệu sinh viên: {data_count}")

# Show sample data
print(f"\n📖 3 dòng dữ liệu mẫu:")
sample_data = data_rdd.take(3)
for i, line in enumerate(sample_data, 1):
    print(f"  {i}: {line}")


Step 2: Đọc file CSV bằng Spark
✅ RDD được tạo với 2358 dòng
✅ Số phân vùng: 2
✅ Không có header, sử dụng tất cả dòng
✅ Số dòng dữ liệu sinh viên: 2358

📖 3 dòng dữ liệu mẫu:
  1: 4351010011,Nguyễn Đình Đô,11/04/2002,Phú Yên,THPT chuyên Lê Quý Đôn,Sư phạm Toán học
  2: 4351010019,Chu Thị Kim Hà,02/09/2002,Phú Yên,THPT Phan Bội Châu,Sư phạm Toán học
  3: 4351010029,Nguyễn thị bích Hồng,12/03/2002,Quảng Ngãi,Trường THPT Lê Trung Đình,Sư phạm Toán học


In [11]:
# Step 3: Phân tích cấu trúc các trường dữ liệu
print("\nStep 3: Phân tích cấu trúc CSV")

# Parse a sample record to understand field positions
sample_record = data_rdd.first()
fields = sample_record.split(',')

print(f"📊 Số trường trong mỗi record: {len(fields)}")
print("📋 Cấu trúc trường (dựa trên dòng mẫu):")

field_names = ["Mã SV", "Họ tên", "Ngày sinh", "Quê quán", "Trường THPT", "Ngành học"]
for i, (name, value) in enumerate(zip(field_names, fields)):
    print(f"  {i}: {name} = '{value}'")

# The hometown field is at index 3 (0-based)
hometown_index = 3
print(f"\n✅ Trường 'Quê quán' ở vị trí index {hometown_index}")
print(f"✅ Giá trị quê quán mẫu: '{fields[hometown_index]}'")

# Show some examples of hometown values
print(f"\n📍 Một số ví dụ về quê quán:")
sample_hometowns = data_rdd.map(lambda line: line.split(',')[hometown_index]).take(10)
for i, hometown in enumerate(sample_hometowns, 1):
    print(f"  {i}: '{hometown}'")


Step 3: Phân tích cấu trúc CSV
📊 Số trường trong mỗi record: 6
📋 Cấu trúc trường (dựa trên dòng mẫu):
  0: Mã SV = '4351010011'
  1: Họ tên = 'Nguyễn Đình Đô'
  2: Ngày sinh = '11/04/2002'
  3: Quê quán = 'Phú Yên'
  4: Trường THPT = 'THPT chuyên Lê Quý Đôn'
  5: Ngành học = 'Sư phạm Toán học'

✅ Trường 'Quê quán' ở vị trí index 3
✅ Giá trị quê quán mẫu: 'Phú Yên'

📍 Một số ví dụ về quê quán:
  1: 'Phú Yên'
  2: 'Phú Yên'
  3: 'Quảng Ngãi'
  4: 'Gia Lai'
  5: 'Bình Định'
  6: 'Phú Yên'
  7: 'Kon Tum'
  8: 'Nghệ An'
  9: ''
  10: 'Bình Định'


In [13]:
# Step 4: Lọc sinh viên có quê quán ở Bình Định
print("\nStep 4: Lọc sinh viên từ Bình Định")

# Filter function to check if student is from Bình Định
def is_from_binh_dinh(line):
    try:
        fields = line.split(',')
        if len(fields) >= 4:  # Ensure we have enough fields
            hometown = fields[3].strip()  # Quê quán field
            return "Bình Định" in hometown
        return False
    except:
        return False

# Apply filter
binh_dinh_students = data_rdd.filter(is_from_binh_dinh)

# Count students from Bình Định
bd_count = binh_dinh_students.count()
print(f"✅ Số sinh viên có quê quán ở Bình Định: {bd_count}")

# Show some examples
print(f"\n👥 10 sinh viên đầu tiên từ Bình Định:")
bd_sample = binh_dinh_students.take(10)
for i, student in enumerate(bd_sample, 1):
    fields = student.split(',')
    name = fields[1] if len(fields) > 1 else "N/A"
    hometown = fields[3] if len(fields) > 3 else "N/A"
    major = fields[5] if len(fields) > 5 else "N/A"
    print(f"  {i}: {name} - {hometown} - {major}")


Step 4: Lọc sinh viên từ Bình Định
✅ Số sinh viên có quê quán ở Bình Định: 1439

👥 10 sinh viên đầu tiên từ Bình Định:
  1: Võ Thụy Ngọc Huyền - Bình Định - Sư phạm Toán học
  2: Lê Thị Thanh Quý - Bình Định - Sư phạm Toán học
  3: Nguyễn Ngọc Như Quỳnh - Bình Định - Sư phạm Toán học
  4: Trần Hoàng Tuyết Sương - Bình Định - Sư phạm Toán học
  5: Ngô Thị Hồng Thắm - Bình Định - Sư phạm Toán học
  6: Lê Nguyễn Anh Thi - Bình Định - Sư phạm Toán học
  7: Lê Nguyễn Anh Thư - Bình Định - Sư phạm Toán học
  8: Nguyễn Anh Thư - Bình Định - Sư phạm Toán học
  9: Đinh Quang Tịnh - Bình Định - Sư phạm Toán học
  10: Đỗ Hữu Tuấn - Bình Định - Sư phạm Toán học


In [15]:
# Step 6: Lưu kết quả và kiểm tra
print("\nStep 6: Lưu kết quả sinh viên Bình Định")

import os, shutil

# Save list of Bình Định students
output_path = "../../results/Bai4_binh_dinh_students"
print(f"💾 Lưu danh sách vào: {output_path}")

# Remove existing output directory
if os.path.exists(output_path):
    shutil.rmtree(output_path)
    print("✅ Đã xóa thư mục output cũ")

# Create results directory
os.makedirs("../../results", exist_ok=True)

# Save the filtered RDD
binh_dinh_students.saveAsTextFile(output_path)
print("✅ Đã lưu danh sách sinh viên Bình Định")

# Verify saved files
if os.path.exists(output_path):
    files = os.listdir(output_path)
    print(f"\n📁 Các file được tạo:")
    for filename in sorted(files):
        filepath = os.path.join(output_path, filename)
        if os.path.isfile(filepath):
            size = os.path.getsize(filepath)
            print(f"  📄 {filename} ({size} bytes)")

    # Read back and verify count
    verification_rdd = sc.textFile(output_path)
    verification_count = verification_rdd.count()
    print(f"\n🔍 Verification: Đọc lại được {verification_count} sinh viên")
    print(f"✅ Khớp với kết quả gốc? {'Đúng' if verification_count == bd_count else 'Sai'}")




Step 6: Lưu kết quả sinh viên Bình Định
💾 Lưu danh sách vào: ../../results/Bai4_binh_dinh_students
✅ Đã xóa thư mục output cũ
✅ Đã lưu danh sách sinh viên Bình Định

📁 Các file được tạo:
  📄 ._SUCCESS.crc (8 bytes)
  📄 .part-00000.crc (624 bytes)
  📄 .part-00001.crc (544 bytes)
  📄 _SUCCESS (0 bytes)
  📄 part-00000 (78623 bytes)
  📄 part-00001 (68286 bytes)

🔍 Verification: Đọc lại được 1439 sinh viên
✅ Khớp với kết quả gốc? Đúng
